In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import robin_stocks.robinhood as r

### Important Links:
https://robin-stocks.readthedocs.io/en/latest/robinhood.html

https://github.com/jmfernandes/robin_stocks/blob/master/Robinhood.rst

In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import robin_stocks.robinhood as r
import yfinance as yf #Yahoo historical data


#u = robinhood username
#p = password for robinhood account
#mfa = mfa from robinhood if 2 auth factor active
#api = api for finnhub
class Stocks:
    def __init__(self, u="", p="", mfa=""):
        self.login = r.login(u, p, mfa_code=mfa)
        self.tickers = []
        self.empty=True
        self.all_update()

                
        if self.empty:
            return
        
        
    def all_update(self):
        self.robinhood_update()
        self.yahoo_update()
        
    def yahoo_update(self):
        self.yTicker = {}
        for i in a.tickers:
            self.yTicker[i] = yf.Ticker(i)
        
        
    def robinhood_update(self):#Frequent running
        prevtickers = self.tickers.sort()
        
        self._base_update_from_robinhood()

        if self.tickers == []:
            self.empty=True
            return
        
        self._adv_update_from_robinhood()
        
        self.empty=False
        #Simple data per stack
        self.quantities = [float(item["quantity"]) for item in self.positions]
        self.boughtAt = [float(item["average_buy_price"]) for item in self.positions]

        self.prevClose = [float(item["previous_close"]) for item in self.tickerInfo]
        self.lastPrice = [float(item["last_trade_price"]) for item in self.tickerInfo]
        self.bidPrice = [float(item["bid_price"]) for item in self.tickerInfo]
        self.askPrice = [float(item["ask_price"]) for item in self.tickerInfo]
        #
        

        #
        
        self._update_daily_stats()
        self._update_total_stats()






    def _base_update_from_robinhood(self):#Information that doesn't need anything else to retrieve
        self.positions = r.get_open_stock_positions()# Per stock | not clean info
        self.profile_info = r.build_user_profile()# Basic Account info
        self.stock_info = r.build_holdings()# Per stock | clean information USEFUL
        self.upcoming_dividends = r.get_dividends() #Upcoming dicidends
        self.tickers = [r.get_symbol_by_url(item["instrument"]) for item in self.positions]# tickers owned
        self.profileData = r.load_portfolio_profile()# Profile info
        self.allTransactions = r.get_bank_transfers()
        self.cardTransactions= r.get_card_transactions()
        self.dividends = r.get_total_dividends()
        
    def _adv_update_from_robinhood(self):#Info from robinhood but needs other info
        self.get_latest_price= r.get_latest_price(self.tickers)#Lastest Price | Have updateable function
        self.get_fundamentals = r.get_fundamentals(self.tickers)# USEFUL Per Stock | Basic infomation about the comapny
        self.tickerInfo =  r.get_quotes(self.tickers)# Per stock | not clean info



        
        
        
        
    def _update_daily_stats(self):#Not needed frequently updated
        #Daily calculations
        self.dailyProfitPerShare = [float(self.lastPrice[i]) - float(self.prevClose[i]) for i in range(len(self.tickers))]
        self.dailyPercentChange = [ 100.0 * self.dailyProfitPerShare[i] / float(self.prevClose[i]) for i in range(len(self.tickers)) ]
        self.dailyProfit = [self.dailyProfitPerShare[i] * self.quantities[i] for i in range(len(self.tickers))]
    
    def _update_total_stats(self):#Not needed frequently updated
        self.deposits = sum(float(x['amount']) for x in self.allTransactions if (x['direction'] == 'deposit') and (x['state'] == 'completed'))
        self.withdrawals = sum(float(x['amount']) for x in self.allTransactions if (x['direction'] == 'withdraw') and (x['state'] == 'completed'))
        self.debits = sum(float(x['amount']['amount']) for x in self.cardTransactions if (x['direction'] == 'debit' and (x['transaction_type'] == 'settled')))
        self.reversal_fees = sum(float(x['fees']) for x in self.allTransactions if (x['direction'] == 'deposit') and (x['state'] == 'reversed'))

        self.money_invested = self.deposits + self.reversal_fees - (self.withdrawals - self.debits)
        self.percentDividend = self.dividends/self.money_invested*100

        self.equity = float(self.profileData['equity'])
        self.totalGainMinusDividends = self.equity - self.dividends - self.money_invested
        self.percentGain = self.totalGainMinusDividends/self.money_invested*100
        
        
        
    def printDailyInfo(self):
        self._update_daily_stats()
        self.dailyTickersPerf = list(zip(self.dailyProfit, self.dailyPercentChange, self.tickers))
        self.dailyTickersPerf.sort(reverse=True)
        print ("My Positions Performance:")
        print ("Ticker | DailyGain | PercentChange")
        for item in self.dailyTickersPerf:
          print ("%s %f$ %f%%" % (item[2], item[0], item[1]))

        print ("Net Gain:", sum(self.dailyProfit)) 
        
        
    
    def logout(self):
        r.logout()

In [48]:
#Pull user name in form .env
# import * from credentials.env
import os
from dotenv import load_dotenv
import pyotp

load_dotenv()

code = os.getenv('MFA')

MFA = pyotp.TOTP(code).now()
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')

api = os.getenv('sandAPI')

In [49]:
a = Stocks(username, password, MFA)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [50]:
a.printDailyInfo()

My Positions Performance:
Ticker | DailyGain | PercentChange
CVX 6.316972$ 1.349423%
OKE 4.623449$ 0.924132%
ARI 3.149864$ 0.523169%
WELL 2.113080$ 1.209247%
IVR 1.743041$ 0.469484%
HAL 1.683185$ 1.275433%
XOM 1.619198$ 0.633304%
SPHD 1.425264$ 1.013662%
NDAQ 1.126373$ 0.238275%
LEG 0.907697$ 0.478342%
DIV 0.407523$ 0.436893%
PGX 0.136519$ 0.292398%
KO 0.122881$ 0.064072%
OHI 0.042329$ 0.035575%
PAYX 0.040000$ 0.033228%
NYMT 0.000000$ 0.000000%
KSS 0.000000$ 0.000000%
TROW -0.060000$ -0.041964%
KBWD -0.121649$ -0.260281%
SDIV -0.361736$ -0.819672%
MSFT -1.318350$ -1.422432%
CLDT -1.350000$ -1.923077%
SRET -1.464660$ -0.784753%
NRZ -1.558580$ -0.572519%
CCL -2.250000$ -3.943218%
CEQP -2.413037$ -1.182486%
AMAT -2.540000$ -1.907193%
MUR -3.511078$ -1.479207%
MITT -3.600000$ -1.128668%
DAL -7.400000$ -3.890641%
FB -13.556647$ -2.469848%
VTI -16.900000$ -0.761776%
Net Gain: -32.94836230000002


In [51]:
a.positions

[{'url': 'https://api.robinhood.com/positions/730028552/d8edff21-cb19-4373-af1c-f83a3c0d01c4/',
  'instrument': 'https://api.robinhood.com/instruments/d8edff21-cb19-4373-af1c-f83a3c0d01c4/',
  'instrument_id': 'd8edff21-cb19-4373-af1c-f83a3c0d01c4',
  'account': 'https://api.robinhood.com/accounts/730028552/',
  'account_number': '730028552',
  'average_buy_price': '7.3770',
  'pending_average_buy_price': '7.3770',
  'quantity': '6.62467500',
  'intraday_average_buy_price': '0.0000',
  'intraday_quantity': '0.00000000',
  'shares_available_for_exercise': '6.62467500',
  'shares_held_for_buys': '0.00000000',
  'shares_held_for_sells': '0.00000000',
  'shares_held_for_stock_grants': '0.00000000',
  'shares_held_for_options_collateral': '0.00000000',
  'shares_held_for_options_events': '0.00000000',
  'shares_pending_from_options_events': '0.00000000',
  'shares_available_for_closing_short_position': '0.00000000',
  'ipo_allocated_quantity': '0.00000000',
  'ipo_dsp_allocated_quantity': '

In [52]:
a.tickerInfo

[{'ask_price': '35.800000',
  'ask_size': 200,
  'bid_price': '34.080000',
  'bid_size': 11,
  'last_trade_price': '35.300000',
  'last_extended_hours_trade_price': '35.300000',
  'previous_close': '35.830000',
  'adjusted_previous_close': '35.830000',
  'previous_close_date': '2022-03-02',
  'symbol': 'MUR',
  'trading_halted': False,
  'has_traded': True,
  'last_trade_price_source': 'consolidated',
  'updated_at': '2022-03-03T22:49:51Z',
  'instrument': 'https://api.robinhood.com/instruments/d8edff21-cb19-4373-af1c-f83a3c0d01c4/',
  'instrument_id': 'd8edff21-cb19-4373-af1c-f83a3c0d01c4',
  'state': 'active'},
 {'ask_price': '33.350000',
  'ask_size': 29,
  'bid_price': '32.300000',
  'bid_size': 60,
  'last_trade_price': '33.350000',
  'last_extended_hours_trade_price': '33.350000',
  'previous_close': '32.930000',
  'adjusted_previous_close': '32.930000',
  'previous_close_date': '2022-03-02',
  'symbol': 'HAL',
  'trading_halted': False,
  'has_traded': True,
  'last_trade_price_

In [53]:
a.profile_info

{'equity': '8889.5389',
 'extended_hours_equity': '8883.5552',
 'cash': '18.38',
 'dividend_total': 482.1800000000001}

In [54]:
a.stock_info

{'MUR': {'price': '35.300000',
  'quantity': '6.62467500',
  'average_buy_price': '7.3770',
  'equity': '233.85',
  'percent_change': '378.51',
  'intraday_percent_change': '0.00',
  'equity_change': '184.980800',
  'type': 'stock',
  'name': 'Murphy Oil',
  'id': 'd8edff21-cb19-4373-af1c-f83a3c0d01c4',
  'pe_ratio': None,
  'percentage': '2.64'},
 'HAL': {'price': '33.350000',
  'quantity': '4.00758400',
  'average_buy_price': '5.4590',
  'equity': '133.65',
  'percent_change': '510.92',
  'intraday_percent_change': '0.00',
  'equity_change': '111.775525',
  'type': 'stock',
  'name': 'Halliburton',
  'id': '38dc15e8-071c-4544-88b3-f633f0a69a4f',
  'pe_ratio': '20.160400',
  'percentage': '1.51'},
 'XOM': {'price': '81.050000',
  'quantity': '3.17489800',
  'average_buy_price': '36.0989',
  'equity': '257.33',
  'percent_change': '124.52',
  'intraday_percent_change': '0.00',
  'equity_change': '142.715157',
  'type': 'stock',
  'name': 'Exxon Mobil',
  'id': '9133b38b-4917-4b5a-8eab-

In [55]:
a.dividends

482.1800000000001

In [56]:
# a.get_notifications

In [57]:
# a.get_news

In [58]:
a.get_latest_price

['35.300000',
 '33.350000',
 '81.050000',
 '62.460000',
 '67.710000',
 '85.370000',
 '46.170000',
 '30.920000',
 '58.990000',
 '3.590000',
 '28.120000',
 '36.730000',
 '8.580000',
 '13.770000',
 '2.120000',
 '10.420000',
 '18.310000',
 '13.450000',
 '156.480000',
 '37.810000',
 '8.760000',
 '142.920000',
 '120.420000',
 '130.640000',
 '296.260000',
 '20.690000',
 '19.160000',
 '13.720000',
 '10.890000',
 '220.160000',
 '203.070000',
 '172.480000']

In [59]:
a.get_fundamentals

[{'open': '35.200000',
  'high': '35.679900',
  'low': '34.690000',
  'volume': '1440635.000000',
  'market_date': '2022-03-03',
  'average_volume_2_weeks': '1991802.818182',
  'average_volume': '1991802.818182',
  'high_52_weeks': '36.530000',
  'dividend_yield': '1.465250',
  'float': '146181157.329000',
  'low_52_weeks': '14.910000',
  'market_cap': '5542261361.557441',
  'pb_ratio': '1.331250',
  'pe_ratio': None,
  'shares_outstanding': '157004571.148936',
  'description': 'Murphy Oil Corp. is a holding company, which engages in the exploration and production of oil and natural gas. It operates through the Exploration and Production and Corporate and Other segment. The Exploration and Production segment includes the United States, Canada, and all other countries. The Corporate and Other segment focuses on interest income, other gains and losses, interest expense, and unallocated overhead. The company was founded by Charles H. Murphy Jr. in 1950 and is headquartered in Houston, TX.

In [60]:
a.profileData

{'url': 'https://api.robinhood.com/portfolios/730028552/',
 'account': 'https://api.robinhood.com/accounts/730028552/',
 'start_date': '2020-03-11',
 'market_value': '8871.1589',
 'equity': '8889.5389',
 'extended_hours_market_value': '8865.1752',
 'extended_hours_equity': '8883.5552',
 'extended_hours_portfolio_equity': '8883.5552',
 'last_core_market_value': '8871.1589',
 'last_core_equity': '8889.5389',
 'last_core_portfolio_equity': '8889.5389',
 'excess_margin': '3865.5826',
 'excess_maintenance': '5713.5989',
 'excess_margin_with_uncleared_deposits': '3865.5826',
 'excess_maintenance_with_uncleared_deposits': '5713.5989',
 'equity_previous_close': '8922.4872',
 'portfolio_equity_previous_close': '8922.4872',
 'adjusted_equity_previous_close': '8922.4872',
 'adjusted_portfolio_equity_previous_close': '8922.4872',
 'withdrawable_amount': '18.38',
 'unwithdrawable_deposits': '0.0000',
 'unwithdrawable_grants': '0.0000',
 'is_primary_account': True}

In [61]:
a.profileData['equity']

'8889.5389'

In [62]:
import time
start = time.time()
start

1646348459.9330778

In [63]:
start

1646348459.9330778

In [64]:
print(60*60*24)

86400


In [65]:
end = time.time()

In [66]:
end

1646348460.0089328

In [67]:
time.gmtime()

time.struct_time(tm_year=2022, tm_mon=3, tm_mday=3, tm_hour=23, tm_min=1, tm_sec=0, tm_wday=3, tm_yday=62, tm_isdst=0)

In [68]:
time.localtime()

time.struct_time(tm_year=2022, tm_mon=3, tm_mday=3, tm_hour=18, tm_min=1, tm_sec=0, tm_wday=3, tm_yday=62, tm_isdst=0)

In [69]:
dic = {'c':123, 'o':120}
dic

{'c': 123, 'o': 120}

In [70]:
dic['ticker'] = 'KO'

In [71]:
dic

{'c': 123, 'o': 120, 'ticker': 'KO'}

In [72]:
import finnhub

# # Setup client
finnhub_client = finnhub.Client(api_key=api)

# # Stock candles
# res = finnhub_client.stock_candles('AAPL', 'D', 1590988249, 1591852249)
# print(res)

# Quote
print(finnhub_client.quote('AAPL'))


# #Convert to Pandas Dataframe
# import pandas as pd
# print(pd.DataFrame(res))

# # Aggregate Indicators
# print(finnhub_client.aggregate_indicator('AAPL', 'D'))

# # Basic financials
# print(finnhub_client.company_basic_financials('AAPL', 'all'))

# # Earnings surprises
# print(finnhub_client.company_earnings('TSLA', limit=5))

# # EPS estimates
# print(finnhub_client.company_eps_estimates('AMZN', freq='quarterly'))

# # Company Executives
# print(finnhub_client.company_executive('AAPL'))

# # Company News
# # Need to use _from instead of from to avoid conflict
# print(finnhub_client.company_news('AAPL', _from="2020-06-01", to="2020-06-10"))

# # Company Peers
# print(finnhub_client.company_peers('AAPL'))

# # Company Profile
# print(finnhub_client.company_profile(symbol='AAPL'))
# print(finnhub_client.company_profile(isin='US0378331005'))
# print(finnhub_client.company_profile(cusip='037833100'))

# # Company Profile 2
# # print(finnhub_client.company_profile2(symbol='AAPL'))

# # Revenue Estimates
# print(finnhub_client.company_revenue_estimates('TSLA', freq='quarterly'))

# # List country
# print(finnhub_client.country())

# # Crypto Exchange
# print(finnhub_client.crypto_exchanges())

# # Crypto symbols
# print(finnhub_client.crypto_symbols('BINANCE'))

# # Economic data
# print(finnhub_client.economic_data('MA-USA-656880'))

# # Filings
# print(finnhub_client.filings(symbol='AAPL', _from="2020-01-01", to="2020-06-11"))

# # Financials
# print(finnhub_client.financials('AAPL', 'bs', 'annual'))

# # Financials as reported
# print(finnhub_client.financials_reported(symbol='AAPL', freq='annual'))

# # Forex exchanges
# print(finnhub_client.forex_exchanges())

# # Forex all pairs
# print(finnhub_client.forex_rates(base='USD'))

# # Forex symbols
# print(finnhub_client.forex_symbols('OANDA'))

# # Fund Ownership
# print(finnhub_client.fund_ownership('AMZN', limit=5))

# # General news
# print(finnhub_client.general_news('forex', min_id=0))

# # Investors ownership
# print(finnhub_client.ownership('AAPL', limit=5))

# # IPO calendar
# print(finnhub_client.ipo_calendar(_from="2020-05-01", to="2020-06-01"))

# # Major developments
# print(finnhub_client.press_releases('AAPL', _from="2020-01-01", to="2020-12-31"))

# # News sentiment
# print(finnhub_client.news_sentiment('AAPL'))

# # Pattern recognition
# print(finnhub_client.pattern_recognition('AAPL', 'D'))

# # Price target
# print(finnhub_client.price_target('AAPL'))

# # Quote
# print(finnhub_client.quote('AAPL'))

# # Recommendation trends
# print(finnhub_client.recommendation_trends('AAPL'))

# # Stock dividends
# print(finnhub_client.stock_dividends('KO', _from='2019-01-01', to='2020-01-01'))

# # Stock dividends 2
# print(finnhub_client.stock_basic_dividends("KO"))

# # Stock symbols
# print(finnhub_client.stock_symbols('US')[0:5])

# # Transcripts
# print(finnhub_client.transcripts('AAPL_162777'))

# # Transcripts list
# print(finnhub_client.transcripts_list('AAPL'))

# # Earnings Calendar
# print(finnhub_client.earnings_calendar(_from="2020-06-10", to="2020-06-30", symbol="", international=False))

# # Covid-19
# print(finnhub_client.covid19())

# # Upgrade downgrade
# print(finnhub_client.upgrade_downgrade(symbol='AAPL', _from='2020-01-01', to='2020-06-30'))

# # Economic code
# print(finnhub_client.economic_code()[0:5])

# # Economic calendar
# print(finnhub_client.calendar_economic('2021-01-01', '2021-01-07'))

# # Support resistance
# print(finnhub_client.support_resistance('AAPL', 'D'))

# # Technical Indicator
# print(finnhub_client.technical_indicator(symbol="AAPL", resolution='D', _from=1583098857, to=1584308457, indicator='rsi', indicator_fields={"timeperiod": 3}))

# # Stock splits
# print(finnhub_client.stock_splits('AAPL', _from='2000-01-01', to='2020-01-01'))

# # Forex candles
# print(finnhub_client.forex_candles('OANDA:EUR_USD', 'D', 1590988249, 1591852249))

# # Crypto Candles
# print(finnhub_client.crypto_candles('BINANCE:BTCUSDT', 'D', 1590988249, 1591852249))

# # Tick Data
# # print(finnhub_client.stock_tick('AAPL', '2020-03-25', 500, 0))

# # BBO Data
# # print(finnhub_client.stock_nbbo("AAPL", "2020-03-25", 500, 0))

# # Indices Constituents
# print(finnhub_client.indices_const(symbol = "^GSPC"))

# # Indices Historical Constituents
# print(finnhub_client.indices_hist_const(symbol = "^GSPC"))

# # ETFs Profile
# print(finnhub_client.etfs_profile('SPY'))
# print(finnhub_client.etfs_profile(isin="US78462F1030"))

# # ETFs Holdings
# print(finnhub_client.etfs_holdings('SPY'))
# print(finnhub_client.etfs_holdings(isin="US00214Q1040", skip=2))

# # ETFs Sector Exposure
# print(finnhub_client.etfs_sector_exp('SPY'))

# # ETFs Country Exposure
# print(finnhub_client.etfs_country_exp('SPY'))

# # International Filings
# print(finnhub_client.international_filings('RY.TO'))
# print(finnhub_client.international_filings(country='GB'))

# # SEC Sentiment Analysis
# print(finnhub_client.sec_sentiment_analysis('0000320193-20-000052'))

# # SEC similarity index
# print(finnhub_client.sec_similarity_index('AAPL'))

# # Bid Ask
# print(finnhub_client.last_bid_ask('AAPL'))

# # FDA Calendar
# print(finnhub_client.fda_calendar())

# # Symbol lookup
# print(finnhub_client.symbol_lookup('apple'))

# # Insider transactions
# print(finnhub_client.stock_insider_transactions('AAPL', '2021-01-01', '2021-03-01'))

# # Mutual Funds Profile
# print(finnhub_client.mutual_fund_profile("VTSAX"))
# print(finnhub_client.mutual_fund_profile(isin="US9229087286"))

# # Mutual Funds Holdings
# print(finnhub_client.mutual_fund_holdings("VTSAX"))
# print(finnhub_client.mutual_fund_holdings(isin="US9229087286", skip=2))

# # Mutual Funds Sector Exposure
# print(finnhub_client.mutual_fund_sector_exp("VTSAX"))

# # Mutual Funds Country Exposure
# print(finnhub_client.mutual_fund_country_exp("VTSAX"))

# # Revenue breakdown
# print(finnhub_client.stock_revenue_breakdown('AAPL'))

# # Social sentiment
# print(finnhub_client.stock_social_sentiment('GME'))

# # Investment Themes
# print(finnhub_client.stock_investment_theme('financialExchangesData'))

# # Supply chain
# print(finnhub_client.stock_supply_chain('AAPL'))

# # Company ESG
# print(finnhub_client.company_esg_score("AAPL"))

# # Earnings Quality Score
# print(finnhub_client.company_earnings_quality_score('AAPL', 'quarterly'))

# # Crypto Profile
# print(finnhub_client.crypto_profile('BTC'))

# # EBITDA Estimates
# print(finnhub_client.company_ebitda_estimates("TSLA", freq="quarterly"))

# # EBIT Estimates
# print(finnhub_client.company_ebit_estimates("TSLA", freq="quarterly"))

# # USPTO Patent
# print(finnhub_client.stock_uspto_patent("AAPL", "2021-01-01", "2021-12-31"))

# # Visa application
# print(finnhub_client.stock_visa_application("AAPL", "2021-01-01", "2022-06-15"))

{'c': 166.23, 'd': -0.33, 'dp': -0.1981, 'h': 168.91, 'l': 165.55, 'o': 168.47, 'pc': 166.56, 't': 1646341202}


In [73]:
import yfinance as yf

msft = yf.Ticker('MSFT')

In [74]:
msft.info

{'zip': '98052-6399',
 'sector': 'Technology',
 'fullTimeEmployees': 181000,
 'longBusinessSummary': 'Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its Productivity and Business Processes segment offers Office, Exchange, SharePoint, Microsoft Teams, Office 365 Security and Compliance, and Skype for Business, as well as related Client Access Licenses (CAL); Skype, Outlook.com, OneDrive, and LinkedIn; and Dynamics 365, a set of cloud-based and on-premises business solutions for organizations and enterprise divisions. Its Intelligent Cloud segment licenses SQL, Windows Servers, Visual Studio, System Center, and related CALs; GitHub that provides a collaboration platform and code hosting service for developers; and Azure, a cloud platform. It also offers support services and Microsoft consulting services to assist customers in developing, deploying, and managing Microsoft server and desktop solutions; and training and certifica

In [75]:
msft.actions

,Dividends,Stock Splits
Date,,
1987-09-21,0.00,2.0
1990-04-16,0.00,2.0
1991-06-27,0.00,1.5
1992-06-15,0.00,1.5
1994-05-23,0.00,2.0
...,...,...
2021-02-17,0.56,0.0
2021-05-19,0.56,0.0
2021-08-18,0.56,0.0


In [76]:
msft.dividends

Date
2003-02-19    0.08
2003-10-15    0.16
2004-08-23    0.08
2004-11-15    3.08
2005-02-15    0.08
              ... 
2021-02-17    0.56
2021-05-19    0.56
2021-08-18    0.56
2021-11-17    0.62
2022-02-16    0.62
Name: Dividends, Length: 73, dtype: float64

In [77]:
dd = yf.download('MSFT')

[*********************100%***********************]  1 of 1 completed


In [78]:
dd

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.061252,1031788800
1986-03-14,0.097222,0.102431,0.097222,0.100694,0.063439,308160000
1986-03-17,0.100694,0.103299,0.100694,0.102431,0.064533,133171200
1986-03-18,0.102431,0.103299,0.098958,0.099826,0.062892,67766400
1986-03-19,0.099826,0.100694,0.097222,0.098090,0.061799,47894400
...,...,...,...,...,...,...
2022-02-25,295.140015,297.630005,291.649994,297.309998,297.309998,32546700
2022-02-28,294.309998,299.140015,293.000000,298.790009,298.790009,34585700
2022-03-01,296.399994,299.970001,292.149994,294.950012,294.950012,31217800


In [79]:
msft = yf.Ticker('MSFT AAPL XOM')

In [80]:
msft.dividends

- MSFT AAPL XOM: No data found, symbol may be delisted


[]

In [81]:
a.tickers

['MUR',
 'HAL',
 'XOM',
 'KO',
 'OKE',
 'WELL',
 'SPHD',
 'CEQP',
 'KSS',
 'NYMT',
 'OHI',
 'DAL',
 'SRET',
 'CLDT',
 'IVR',
 'NRZ',
 'CCL',
 'ARI',
 'CVX',
 'LEG',
 'MITT',
 'TROW',
 'PAYX',
 'AMAT',
 'MSFT',
 'DIV',
 'KBWD',
 'PGX',
 'SDIV',
 'VTI',
 'FB',
 'NDAQ']

In [120]:
info = {}

for i in a.tickers:
    info[i] = yf.Ticker(i)
    h = pd.DataFrame(info[i].history())
    h['Date'] = h.index
    info[i].history = h
    

In [118]:
h.index

DatetimeIndex(['2022-02-04', '2022-02-07', '2022-02-08', '2022-02-09',
               '2022-02-10', '2022-02-11', '2022-02-14', '2022-02-15',
               '2022-02-16', '2022-02-17', '2022-02-18', '2022-02-22',
               '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-28',
               '2022-03-01', '2022-03-02', '2022-03-03'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [122]:
info['XOM'].history

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
Date,,,,,,,,
2022-02-04,79.983315,81.625110,79.983315,80.517395,32176400,0.00,0,2022-02-04
2022-02-07,80.250346,82.169078,79.755829,81.486641,34949200,0.00,0,2022-02-07
2022-02-08,81.367953,81.595435,79.043721,79.379997,34356300,0.00,0,2022-02-08
2022-02-09,79.580002,80.029999,78.709999,79.000000,40106100,0.88,0,2022-02-09
2022-02-10,78.720001,79.680000,77.790001,78.239998,28527700,0.00,0,2022-02-10
2022-02-11,78.620003,80.320000,78.230003,80.209999,41969100,0.00,0,2022-02-11
2022-02-14,79.580002,79.839996,77.050003,78.980003,34652800,0.00,0,2022-02-14
2022-02-15,76.900002,78.190002,76.419998,77.989998,25075100,0.00,0,2022-02-15
2022-02-16,78.790001,79.730003,78.040001,78.349998,21733800,0.00,0,2022-02-16


In [84]:
a.yHist['XOM']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1962-01-02,0.000000,1.589844,1.578125,1.578125,0.101854,902400
1962-01-03,0.000000,1.601563,1.578125,1.601563,0.103367,1200000
1962-01-04,0.000000,1.613281,1.597656,1.605469,0.103619,1088000
1962-01-05,0.000000,1.613281,1.566406,1.570313,0.101350,1222400
1962-01-08,0.000000,1.582031,1.546875,1.566406,0.101098,1388800
...,...,...,...,...,...,...
2022-02-25,76.690002,77.989998,76.260002,77.839996,77.839996,28310300
2022-02-28,76.510002,78.930000,76.199997,78.419998,78.419998,32187600
2022-03-01,78.769997,80.650002,78.330002,79.169998,79.169998,40002300


In [94]:
a.yTicker['MUR'].history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-03-17,0.000000,2.051803,1.912878,1.912878,148687,0.0,0.0
1980-03-18,0.000000,1.944937,1.875475,1.944937,451620,0.0,0.0
1980-03-19,0.000000,2.025087,1.966312,2.014401,245959,0.0,0.0
1980-03-20,0.000000,2.009057,1.966311,1.971654,266803,0.0,0.0
1980-03-21,0.000000,1.966311,1.875476,1.880819,309881,0.0,0.0
...,...,...,...,...,...,...,...
2022-02-25,33.040001,33.200001,32.000000,33.160000,2171100,0.0,0.0
2022-02-28,32.860001,35.119999,32.820000,34.669998,2338800,0.0,0.0
2022-03-01,35.220001,36.000000,34.570000,35.250000,2783000,0.0,0.0


In [92]:
a.yTicker['MUR'].dividends

Date
1983-05-10    0.053972
1983-08-09    0.053972
1983-11-07    0.053972
1984-02-08    0.053972
1984-05-09    0.053972
                ...   
2021-02-12    0.125000
2021-05-14    0.125000
2021-08-13    0.125000
2021-11-12    0.125000
2022-02-11    0.150000
Name: Dividends, Length: 157, dtype: float64

In [95]:
a.yTicker['MUR'].financials

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,42891000.0,-1549035000.0,203498000.0,43002000.0
Minority Interest,163485000.0,179810000.0,337151000.0,368343000.0
Net Income,-73664000.0,-1148777000.0,1149732000.0,411094000.0
Selling General Administrative,360437000.0,388462000.0,232736000.0,205192000.0
Gross Profit,2271014000.0,1197933000.0,1987657000.0,1325526000.0
Ebit,516962000.0,-132785000.0,441167000.0,203874000.0
Operating Income,516962000.0,-132785000.0,441167000.0,203874000.0
Other Operating Expenses,530368000.0,-130468000.0,70807000.0,39034000.0


In [96]:
a.yTicker['MUR'].major_holders

,0,1
0,6.30%,% of Shares Held by All Insider
1,80.53%,% of Shares Held by Institutions
2,85.94%,% of Float Held by Institutions
3,388,Number of Institutions Holding Shares


In [97]:
a.yTicker['MUR'].cashflow

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Change To Liabilities,9.516600e+07,-2.087400e+08,1.967730e+08,8.784000e+06
Total Cashflows From Investing Activities,-4.177100e+08,-8.590200e+08,-2.536204e+09,-1.804740e+09
Net Borrowings,-5.352480e+08,1.864670e+08,-3.046260e+08,3.246820e+08
Total Cash From Financing Activities,-7.945130e+08,3.971100e+07,-1.129994e+09,1.435620e+08
Change To Operating Activities,2.003000e+06,7.178000e+06,4.650000e+06,-1.001100e+07
Net Income,-7.366400e+07,-1.148777e+09,1.149732e+09,4.110940e+08
Change In Cash,2.105780e+08,3.846000e+06,-5.316300e+07,-2.705100e+08
Repurchase Of Stock,-5.209000e+06,-7.094000e+06,-5.069150e+08,-8.076000e+06
Effect Of Exchange Rate,6.380000e+05,2.009000e+06,3.533000e+06,2.873000e+07
Total Cash From Operating Activities,1.422163e+09,8.027080e+08,1.489105e+09,7.493950e+08


In [98]:
a.yTicker['MUR'].earnings

,Revenue,Earnings
Year,,
2018,1806473000,411094000
2019,2817111000,1149732000
2020,1751709000,-1148777000
2021,2801215000,-73664000


In [99]:
a.yTicker['MUR'].quarterly_earnings

,Revenue,Earnings
Quarter,,
1Q2021,592527000,-287438000
2Q2021,758829000,-63082000
3Q2021,687549000,108456000
4Q2021,762310000,168400000


In [101]:
a.yTicker['MUR'].recommendations

,Firm,To Grade,From Grade,Action
Date,,,,
2012-02-14 06:38:00,Brean Capital,Buy,,init
2012-02-29 06:05:00,Deutsche Bank,Hold,,down
2012-05-03 10:33:00,UBS,Neutral,,main
2012-05-07 06:18:00,Brean Capital,Hold,,down
2012-05-07 08:18:00,Societe Generale,Buy,,up
...,...,...,...,...
2021-12-07 10:49:48,JP Morgan,Overweight,Neutral,up
2022-01-05 11:18:14,Wells Fargo,Equal-Weight,Underweight,up
2022-01-25 15:24:11,Morgan Stanley,Equal-Weight,,main


In [103]:
a.yTicker['MUR'].calendar

,0,1
Earnings Date,2022-05-04 12:30:00,2022-05-09 12:30:00
Earnings Average,0.47,0.47
Earnings Low,0.18,0.18
Earnings High,0.77,0.77
Revenue Average,607600000,607600000
Revenue Low,531000000,531000000
Revenue High,691000000,691000000


In [104]:
a.yTicker['MUR'].news

[{'uuid': 'd58e2398-8f5c-3a63-9e53-11513c47ada9',
  'title': 'This Energy Stock Just Boosted Its Dividend by 20%. Are More Hikes on the Way?',
  'publisher': 'Motley Fool',
  'link': 'https://finance.yahoo.com/m/d58e2398-8f5c-3a63-9e53-11513c47ada9/this-energy-stock-just.html',
  'providerPublishTime': 1645191300,
  'type': 'STORY'},
 {'uuid': '74758bf3-dcd0-3b3f-97cf-db1ba341ae8a',
  'title': "Murphy Oil's (NYSE:MUR) Dividend Will Be Increased To US$0.15",
  'publisher': 'Simply Wall St.',
  'link': 'https://finance.yahoo.com/news/murphy-oils-nyse-mur-dividend-135122009.html',
  'providerPublishTime': 1644414682,
  'type': 'STORY'},
 {'uuid': '7e29a18f-0417-3292-9b30-9348011a7c35',
  'title': 'Affinity Wealth Management Llc Buys Fidelity Total Bond ETF, First Trust Dorsey Wright Focus 5 ...',
  'publisher': 'GuruFocus.com',
  'link': 'https://finance.yahoo.com/news/affinity-wealth-management-llc-buys-143822773.html',
  'providerPublishTime': 1644331102,
  'type': 'STORY'},
 {'uuid': '

In [105]:
pd.read_csv('https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Bootstrap/Side-Bar/iranian_students.csv')

,Years,Girls Kindergarten,Boys Kindergarten,Girls Grade School,Boys Grade School,Girls Middle School,Boys Middle School,Girls High School,Boys High School
0,1971,9509,11728,1069331,1734310,97464,161754,390881,797565
1,1972,10025,11748,1137189,1825407,210381,361084,330668,668600
2,1973,18587,22400,1180938,1957533,330293,576045,269295,547629
3,1974,40574,42280,1348350,2173074,411746,739710,273383,524235
4,1975,79978,95446,1491431,2321089,459419,824242,297107,558411
5,1976,97434,114435,1619235,2459239,493394,875516,332835,608754
6,1977,108298,126838,1732580,2581835,528769,917571,354566,634712
7,1978,119662,139835,1769630,2625342,563126,972137,393750,693809
8,1979,109123,127363,1941547,3067778,626443,1063448,438830,674398
9,1980,81163,90840,1899506,2899086,599705,975359,416536,626861


In [210]:
import yfinance as yf
dsfliveData = yf.download("AAPL", period="7d", interval="1m", prepost=False, threads=True)

[*********************100%***********************]  1 of 1 completed


In [211]:
dsfliveData['Date'] = dsfliveData.index

In [212]:
dsfliveData.head(500)

,Open,High,Low,Close,Adj Close,Volume,Date
Datetime,,,,,,,
2022-02-24 09:30:00-05:00,152.580002,153.100006,152.000000,152.979996,152.979996,13403194,2022-02-24 09:30:00-05:00
2022-02-24 09:31:00-05:00,153.000000,153.110306,152.600006,152.837997,152.837997,1355205,2022-02-24 09:31:00-05:00
2022-02-24 09:32:00-05:00,152.850006,154.169998,152.824997,154.020004,154.020004,1467634,2022-02-24 09:32:00-05:00
2022-02-24 09:33:00-05:00,154.029999,154.149994,153.741394,154.044998,154.044998,1045341,2022-02-24 09:33:00-05:00
2022-02-24 09:34:00-05:00,153.589996,153.800995,153.529999,153.800995,153.800995,945750,2022-02-24 09:34:00-05:00
...,...,...,...,...,...,...,...
2022-02-25 11:16:00-05:00,164.009995,164.059998,163.880005,163.889999,163.889999,256983,2022-02-25 11:16:00-05:00
2022-02-25 11:17:00-05:00,163.895493,163.929993,163.740005,163.835007,163.835007,204117,2022-02-25 11:17:00-05:00
2022-02-25 11:18:00-05:00,163.839996,164.029999,163.830002,164.005005,164.005005,161281,2022-02-25 11:18:00-05:00


In [213]:
from datetime import time
df = dsfliveData


In [216]:
import plotly.express as px

fig = px.line(dsfliveData, x="Date", y="Close", title="Live Price").update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=5, label="5m", step="minute", stepmode="backward"),
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=5, label="5d", step="day", stepmode="backward"),
            dict(count=1, label="1w", step="all", stepmode="todate"),

        ])
    )
)

In [217]:
fig.show()

In [202]:
fig = px.line(df, x="Date", y="Close", title="Live Price").update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=5, label="5m", step="minute", stepmode="backward"),
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=5, label="5d", step="day", stepmode="backward"),
            dict(count=1, label="1w", step="all", stepmode="todate"),

        ])
    )
)
fig.show()

In [206]:
import plotly.graph_objects as go
from datetime import datetime
import pandas as pd
import numpy as np

# sample data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

# remove some dates to build a similar case as in the question
df = df.drop(df.index[75:110])
df = df.drop(df.index[210:250])
df = df.drop(df.index[460:480])

# build complete timepline from start date to end date
dt_all = pd.date_range(start=df['Date'].iloc[0],end=df['Date'].iloc[-1])
print(dt_all)
# retrieve the dates that ARE in the original datset
dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df['Date'])]
print(dt_obs)
# define dates with missing values
dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
print()
print(dt_breaks)
# make fiuge
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'])
                      ])

# hide dates with no values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

fig.update_layout(yaxis_title='AAPL Stock')

fig.show()

DatetimeIndex(['2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20',
               '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24',
               '2015-02-25', '2015-02-26',
               ...
               '2017-02-07', '2017-02-08', '2017-02-09', '2017-02-10',
               '2017-02-11', '2017-02-12', '2017-02-13', '2017-02-14',
               '2017-02-15', '2017-02-16'],
              dtype='datetime64[ns]', length=731, freq='D')
['2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19', '2015-03-20', '2015-03-23', '2015-03-24', '2015-03-25', '2015-03-26', '2015-03-27', '2015-03-30', '2015-03-31', '2015-04-01', '2015-04-02', '2015-04-06', '2015-04-07', '2015-04-08', '2015-04-09', '2015-04-10', '2015-04-13', '20

KeyError: 'Open'